In [1]:
import pickle
import pandas as pd
import requests
import json
import time
import os
import time
import pandas as pd

In [2]:
with open("congress_members.pkl", "rb") as file:
    congress_members_df = pickle.load(file)

In [3]:
filtered_df = congress_members_df[(congress_members_df["start"] <= 2023) & (congress_members_df["end"].fillna(2023) >= 2013)]
unique_bioguide_ids = filtered_df["bioguideId"].unique()

In [4]:
YOUR_API_KEY = ''

def get_sponsored_legislation(bioguide_id, offset):
    url = f"https://api.congress.gov/v3/member/{bioguide_id}/sponsored-legislation?api_key={YOUR_API_KEY}&format=json&offset={offset}&limit=250"
    response = requests.get(url)
    return response.json()

In [5]:
def parse_legislation_data(legislation, bioguide_id):
    return {
        'bioguide_id': bioguide_id,
        'congress': legislation.get('congress', None),
        'introduced_date': legislation.get('introducedDate', None),
        'number': legislation.get('number', None),
        'title': legislation.get('title', None),
        'type': legislation.get('type', None),
    }


In [6]:
def save_to_csv(legislation_data, batch_number):
    legislation_df = pd.DataFrame(legislation_data)
    legislation_df.to_csv(f"legislation_data_batch_{batch_number}.csv", index=False)
    print(f"Saved legislation_data_batch_{batch_number}.csv")

In [7]:
legislation_data = []
batch_number = 1
bioguide_ids_processed = 0

api_calls = 0
start_time = time.time()

for index, bioguide_id in enumerate(unique_bioguide_ids, start=1):
    offset = 0

    while True:
        if api_calls >= 1000:
            elapsed_time = time.time() - start_time
            if elapsed_time < 3600:  # 3600 seconds in an hour
                time_to_wait = 3600 - elapsed_time
                time.sleep(time_to_wait)

            # Reset API call count and start time after waiting
            api_calls = 0
            start_time = time.time()

        sponsored_legislation_data = get_sponsored_legislation(bioguide_id, offset)
        api_calls += 1
        sponsored_legislation = sponsored_legislation_data.get("sponsoredLegislation", [])

        if not sponsored_legislation:
            break

        for legislation in sponsored_legislation:
            legislation_data.append(parse_legislation_data(legislation, bioguide_id))

        offset += 250

    bioguide_ids_processed += 1
    print(f"Progress: {index}/{len(unique_bioguide_ids)} - Bioguide ID: {bioguide_id}")

    if bioguide_ids_processed % 50 == 0:
        save_to_csv(legislation_data, batch_number)
        legislation_data = []
        batch_number += 1

# Save remaining data after the loop
if legislation_data:
    save_to_csv(legislation_data, batch_number)

Progress: 1/1044 - Bioguide ID: W000790
Progress: 2/1044 - Bioguide ID: B001319
Progress: 3/1044 - Bioguide ID: H001086
Progress: 4/1044 - Bioguide ID: L000593
Progress: 5/1044 - Bioguide ID: O000174
Progress: 6/1044 - Bioguide ID: C001098
Progress: 7/1044 - Bioguide ID: B001305
Progress: 8/1044 - Bioguide ID: W000817
Progress: 9/1044 - Bioguide ID: W000802
Progress: 10/1044 - Bioguide ID: Y000064
Progress: 11/1044 - Bioguide ID: W000800
Progress: 12/1044 - Bioguide ID: W000779
Progress: 13/1044 - Bioguide ID: W000437
Progress: 14/1044 - Bioguide ID: V000137
Progress: 15/1044 - Bioguide ID: T000278
Progress: 16/1044 - Bioguide ID: T000476
Progress: 17/1044 - Bioguide ID: W000805
Progress: 18/1044 - Bioguide ID: T000250
Progress: 19/1044 - Bioguide ID: V000128
Progress: 20/1044 - Bioguide ID: S001203
Progress: 21/1044 - Bioguide ID: S001198
Progress: 22/1044 - Bioguide ID: S001191
Progress: 23/1044 - Bioguide ID: T000464
Progress: 24/1044 - Bioguide ID: S001184
Progress: 25/1044 - Biogu

Progress: 197/1044 - Bioguide ID: D000627
Progress: 198/1044 - Bioguide ID: M001202
Progress: 199/1044 - Bioguide ID: L000586
Progress: 200/1044 - Bioguide ID: O000171
Saved legislation_data_batch_4.csv
Progress: 201/1044 - Bioguide ID: S001197
Progress: 202/1044 - Bioguide ID: K000386
Progress: 203/1044 - Bioguide ID: R000602
Progress: 204/1044 - Bioguide ID: Z000017
Progress: 205/1044 - Bioguide ID: L000581
Progress: 206/1044 - Bioguide ID: H001071
Progress: 207/1044 - Bioguide ID: R000597
Progress: 208/1044 - Bioguide ID: M001185
Progress: 209/1044 - Bioguide ID: W000813
Progress: 210/1044 - Bioguide ID: B001286
Progress: 211/1044 - Bioguide ID: D000619
Progress: 212/1044 - Bioguide ID: L000579
Progress: 213/1044 - Bioguide ID: S001175
Progress: 214/1044 - Bioguide ID: S001165
Progress: 215/1044 - Bioguide ID: R000585
Progress: 216/1044 - Bioguide ID: M001180
Progress: 217/1044 - Bioguide ID: H001056
Progress: 218/1044 - Bioguide ID: G000563
Progress: 219/1044 - Bioguide ID: P000601

Progress: 389/1044 - Bioguide ID: S001215
Progress: 390/1044 - Bioguide ID: S001208
Progress: 391/1044 - Bioguide ID: T000483
Progress: 392/1044 - Bioguide ID: P000617
Progress: 393/1044 - Bioguide ID: T000482
Progress: 394/1044 - Bioguide ID: D000629
Progress: 395/1044 - Bioguide ID: W000824
Progress: 396/1044 - Bioguide ID: P000615
Progress: 397/1044 - Bioguide ID: B001307
Progress: 398/1044 - Bioguide ID: U000040
Progress: 399/1044 - Bioguide ID: C001117
Progress: 400/1044 - Bioguide ID: G000586
Saved legislation_data_batch_8.csv
Progress: 401/1044 - Bioguide ID: F000469
Progress: 402/1044 - Bioguide ID: F000467
Progress: 403/1044 - Bioguide ID: M001208
Progress: 404/1044 - Bioguide ID: S001206
Progress: 405/1044 - Bioguide ID: M001207
Progress: 406/1044 - Bioguide ID: S001214
Progress: 407/1044 - Bioguide ID: S001210
Progress: 408/1044 - Bioguide ID: W000823
Progress: 409/1044 - Bioguide ID: H001081
Progress: 410/1044 - Bioguide ID: C001121
Progress: 411/1044 - Bioguide ID: N000191

KeyboardInterrupt: 

In [ ]:
with open("sponsored_legislation.pkl", "wb") as file:
    pickle.dump(legislation_df, file)